In [ ]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))


In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

In [1]:
import os
import cv2
import yaml

# Base dataset path
base_path = 'Datasets/Preprpcess_Large_Dataset'
splits = ['train', 'val', 'test']

# Get class names from train/images subfolders
train_images_path = os.path.join(base_path, 'train', 'images')
class_names = sorted([d for d in os.listdir(train_images_path) if os.path.isdir(os.path.join(train_images_path, d))])
class_to_id = {name: idx for idx, name in enumerate(class_names)}

def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Process each split
for split in splits:
    for class_name in class_names:
        image_dir = os.path.join(base_path, split, 'images', class_name)
        label_dir = os.path.join(base_path, split, 'labels', class_name)
        ensure_dir(label_dir)

        if not os.path.exists(image_dir):
            print(f"Skipping missing folder: {image_dir}")
            continue

        for img_file in os.listdir(image_dir):
            if not img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue

            img_path = os.path.join(image_dir, img_file)
            image = cv2.imread(img_path)
            if image is None:
                print(f"Warning: couldn't read image {img_path}")
                continue

            h, w, _ = image.shape

            # Dummy bounding box: center of image, 50% size
            x_center = 0.5
            y_center = 0.5
            bbox_width = 0.5
            bbox_height = 0.5
            class_id = class_to_id[class_name]

            # Write YOLO label file
            label_path = os.path.join(label_dir, os.path.splitext(img_file)[0] + '.txt')
            with open(label_path, 'w') as f:
                f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")

print("✅ Labels generated for all splits.")

# Create dataset.yaml file
yaml_data = {
    'path': os.path.abspath(base_path),
    'train': os.path.abspath(os.path.join(base_path, 'train')),
    'val': os.path.abspath(os.path.join(base_path, 'val')),
    'test': os.path.abspath(os.path.join(base_path, 'test')),
    'nc': len(class_names),
    'names': class_names
}

with open('emotion_dataset.yaml', 'w') as f:
    yaml.dump(yaml_data, f, default_flow_style=False)

print("✅ 'emotion_dataset.yaml' created.")


Skipping missing folder: Datasets/Preprpcess_Large_Dataset\val\images\angry
Skipping missing folder: Datasets/Preprpcess_Large_Dataset\val\images\boring
Skipping missing folder: Datasets/Preprpcess_Large_Dataset\val\images\disgust
Skipping missing folder: Datasets/Preprpcess_Large_Dataset\val\images\fear
Skipping missing folder: Datasets/Preprpcess_Large_Dataset\val\images\happy
Skipping missing folder: Datasets/Preprpcess_Large_Dataset\val\images\neutral
Skipping missing folder: Datasets/Preprpcess_Large_Dataset\val\images\sad
Skipping missing folder: Datasets/Preprpcess_Large_Dataset\val\images\stress
Skipping missing folder: Datasets/Preprpcess_Large_Dataset\val\images\suprised
Skipping missing folder: Datasets/Preprpcess_Large_Dataset\test\images\angry
Skipping missing folder: Datasets/Preprpcess_Large_Dataset\test\images\boring
Skipping missing folder: Datasets/Preprpcess_Large_Dataset\test\images\disgust
Skipping missing folder: Datasets/Preprpcess_Large_Dataset\test\images\fear


In [5]:
import os
import shutil
import random

# Base dataset path
base_dir = 'Datasets/SampleDatasetNew'

# Ratios
split_ratios = {
    'train': 0.7,
    'val': 0.15,
    'test': 0.15
}

# Source folders
image_root = os.path.join(base_dir, 'train', 'images')
label_root = os.path.join(base_dir, 'train', 'labels')

# Classes (subfolders)
class_names = [d for d in os.listdir(image_root) if os.path.isdir(os.path.join(image_root, d))]

for cls in class_names:
    img_class_dir = os.path.join(image_root, cls)
    lbl_class_dir = os.path.join(label_root, cls)

    images = [f for f in os.listdir(img_class_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    random.shuffle(images)

    total = len(images)
    n_train = int(split_ratios['train'] * total)
    n_val = int(split_ratios['val'] * total)

    split_map = {
        'train': images[:n_train],
        'val': images[n_train:n_train + n_val],
        'test': images[n_train + n_val:]
    }

    for split, files in split_map.items():
        img_dest_dir = os.path.join(base_dir, split, 'images', cls)
        lbl_dest_dir = os.path.join(base_dir, split, 'labels', cls)

        os.makedirs(img_dest_dir, exist_ok=True)
        os.makedirs(lbl_dest_dir, exist_ok=True)

        moved = 0
        for file in files:
            base_name, _ = os.path.splitext(file)
            label_file = base_name + '.txt'

            img_src = os.path.join(img_class_dir, file)
            lbl_src = os.path.join(lbl_class_dir, label_file)

            img_dst = os.path.join(img_dest_dir, file)
            lbl_dst = os.path.join(lbl_dest_dir, label_file)

            if os.path.exists(img_src):
                shutil.move(img_src, img_dst)
            else:
                print(f"❌ Missing image: {img_src}")
                continue

            if os.path.exists(lbl_src):
                shutil.move(lbl_src, lbl_dst)
            else:
                print(f"⚠️ Missing label: {lbl_src}")

            moved += 1

        print(f"✅ {split.upper()} - {cls}: Moved {moved} images.")

print("\n🎉 Dataset split complete with subfolders preserved.")


✅ TRAIN - angry: Moved 770 images.
✅ VAL - angry: Moved 165 images.
✅ TEST - angry: Moved 165 images.
✅ TRAIN - disgust: Moved 770 images.
✅ VAL - disgust: Moved 165 images.
✅ TEST - disgust: Moved 165 images.
✅ TRAIN - fear: Moved 770 images.
✅ VAL - fear: Moved 165 images.
✅ TEST - fear: Moved 165 images.
✅ TRAIN - happy: Moved 766 images.
✅ VAL - happy: Moved 164 images.
✅ TEST - happy: Moved 165 images.
✅ TRAIN - neutral: Moved 770 images.
✅ VAL - neutral: Moved 165 images.
✅ TEST - neutral: Moved 165 images.
✅ TRAIN - sad: Moved 770 images.
✅ VAL - sad: Moved 165 images.
✅ TEST - sad: Moved 165 images.
✅ TRAIN - suprise: Moved 770 images.
✅ VAL - suprise: Moved 165 images.
✅ TEST - suprise: Moved 165 images.

🎉 Dataset split complete with subfolders preserved.


In [1]:
!pip install -U ultralytics

  Using cached nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.6.4.1-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.3.0.4-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.7.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.7.1.2-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.5.4.2-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_6

In [1]:


from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)

# Train using the single most idle GPU
results = model.train(data="Datasets/SampleDatasetNew", epochs=100, imgsz=360, device=0,batch=64,pretrained=True)

New https://pypi.org/project/ultralytics/8.3.144 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.133  Python-3.9.21 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Datasets/SampleDatasetNew, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=360, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train7, nbs=64, nms=False, ops

c:\Users\LOQ\anaconda3\envs\pythonenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-26 00:56:49,803	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-05-26 00:56:50,707	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning D:\RuhunaNew\Academic\Research\Facial_Recog\Group_50\Datasets\SampleDatasetNew\train... 5386 images, 0 corrupt: 100%|██████████| 5386/5386 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.00.0 ms, read: 337.2269.4 MB/s, size: 436.2 KB)


...

     11/100      2.96G  2.905e-06         10        384: 100%|██████████| 85/85 [00:11<00:00,  7.64it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 10/10 [00:00<00:00, 15.40it/s]

                   all          1          1



      Epoch    GPU_mem       loss  Instances       Size


     12/100      2.96G  2.772e-06         64        384:  14%|█▍        | 12/85 [00:01<00:11,  6.28it/s]


KeyboardInterrupt: 

In [ ]:
model.export(format="onnx")

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("E:\AppsTechnologies\AI\BeMA\Facial_Recog\\best.pt")  # pretrained YOLO11n model

# Run batched inference on a list of images
results = model(["E:\AppsTechnologies\AI\BeMA\Facial_Recog\img.jpg"])  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk

In [ ]:
import cv2
import time
from ultralytics import YOLO

# Define your emotion class names
emotion_classes = ['angry','boring','disgust','fear','happy','neutral','sad','stress','surprise']

# Load your custom model
model = YOLO("D:\RuhunaNew\Academic\Research\Facial_Recog\Group_50\\best.pt")

cap = cv2.VideoCapture(0)
inference_interval = 5  # seconds
last_inference_time = 0
last_label = "No emotion detected"

while True:
    ret, frame = cap.read()
    if not ret:
        break

    current_time = time.time()
    # Run inference every 5 seconds
    if current_time - last_inference_time >= inference_interval:
        results = model(frame)
        last_inference_time = current_time

        # Process results
        if results[0].boxes is not None and len(results[0].boxes) > 0:
            box = results[0].boxes[0]
            class_id = int(box.cls[0])
            confidence = box.conf[0].item()
            last_label = f"{emotion_classes[class_id]} ({confidence:.2f})"
            # Optionally, draw box:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(frame, last_label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
        else:
            last_label = "No emotion detected"

    # Overlay the last detected label on the frame
    cv2.putText(frame, last_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("YOLOv11 Emotion Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLO11n model
model = YOLO("D:\\RuhunaNew\\Academic\\Research\\Facial_Recog\\best.pt")

# Run inference on the source
results = model(source=0, stream=True)  # generator of Results objects

# Process the results
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    print(boxes)